Una vez que mi DF tiene como máximo 1 millón de rows, me conviene trabajar con Pandas DataFrames
 - Porque es parecido a R.
 - Correr funciones es más rápido que hacer una UDF (user defined function) en un Spark DataFrame

### Levantar un csv como Pandas DataFrame

In [ ]:
import pandas as pd
import numpy as np

#### pandas.read_csv nos da muchas opciones: 
 - sep:"," puedo decirle el separador
 - header='true' puedo decirle si mi tabla tiene header
 - parse_dates=['action_date'] puedo decirle que me pase a formato fecha los campos de fecha por nombre o posicion
 - names=['nuevo_nombre1', 'nuevo_nombre2' ... 'nuevo_nombreN']  puedo decirle ahi mismo los nuevos nombres de las columnas
 - dtype={'a': np.float64} puedo decir uno por uno el tipo de variable
 - mas opciones!

In [ ]:
PDF = pd.read_csv("PandasDF_sample.csv", sep=',', header='infer')

### Exploración de datos

#### Pedir un head() y tail()

In [ ]:
PDF.head()

In [ ]:
PDF.tail()

#### Ver solo las primeras N filas

In [ ]:
PDF.iloc[:3]

In [ ]:
# es lo mismo hacer
PDF[:3]

#### Preguntar los nombres de las columnas

In [ ]:
PDF.columns.values

#### Pedir un summary de las variables numéricas

In [ ]:
# todas las variables numéricas
PDF.describe()

In [ ]:
# summary de un subgrupo que cumple cierta condicion
PDF[PDF['fl']=="thanks"].describe()

In [ ]:
# podemos pedir determinados percentiles para incluir en el describe
PDF.describe(percentiles=[.02, .05, 0.95, 0.99])

#### Table de un campo

In [ ]:
PDF['fl'].value_counts()

#### Table de una combinación de campos

In [ ]:
PDF.groupby(["traveler_type", "destination_type"]).size().reset_index(name="Freq")

#### Operaciones con campos

In [ ]:
PDF['gb_night'] = PDF['pri'] / PDF['duration']

#### Llenar missing values

In [ ]:
PDF['gb_night'] = PDF['gb_night'].fillna(0)

#### Aplicar una función con apply
- sobre un campo

In [ ]:
# Si comprador = True, poner 1, si no poner 0
PDF['comprador'] = PDF['comprador'].apply(lambda x: 1 if (x == True) else 0)

- sobre varios campos de un mismo row

In [ ]:
### RATIO FINDE###
# Define udf
#### cuenta como finde las noches de ci viernes,sábado y domingo.
def calcRatioFinde(ci, duracion):
    
    if ci is None:
        return None
    
    dia_ci = ci.isoweekday()
    # cuantas semanas completas de 7 dias (con dos dias de finde) hay en la duracion del viaje?
    # trunc(duracion/7)
    semanas_completas = int(duracion/7)
    
    # el resto de los dias fuera de las semanas completas:
    resto_dias = duracion - semanas_completas * 7
   
    # cuento los dias de fin de semana extras a la semana completa
    if dia_ci == 5: # si el grupo de dias extra empieza un viernes, como maximo pueden contener vie, sab y dom (3 dias) de finde
        dia_finde_extra = min(3, resto_dias) 
    elif dia_ci == 6:  # si el grupo de dias extra empieza un sabado, como maximo pueden contener sab y dom (2 dias) de finde
        dia_finde_extra = min(2, resto_dias) 
    elif dia_ci == 7:  # si el grupo de dias extra empieza un domingo, si son 6 dias extra contienen un domingo un un viernes
        if resto_dias == 6:
            dia_finde_extra = 2
        else:
            dia_finde_extra = 1
    elif dia_ci + resto_dias == 6: # abarca todos los casos en que el grupo de dias extra  empieza lu/ma/mi/ju y termina en vi
        dia_finde_extra = 1
    elif dia_ci + resto_dias == 7: # abarca todos los casos en que el grupo de dias extra  empieza lu/ma/mi/ju y termina en sa
        dia_finde_extra = 2
    elif dia_ci + resto_dias > 8: # abarca los casos en que el grupo de dias extra empieza lu/ma/mi/ju e incluye un dom
        dia_finde_extra = 3
    else:
        dia_finde_extra = 0
    
    # Saco el numero de dias de finde (dos por semana completa, mas los dias extra)
    # el round es porque python, si divide integers, redondea a integer la respuesta
    dias_finde_total = semanas_completas * 2 + round(dia_finde_extra,2)
    
    # saco el ratio "dias de finde" / "duracion" del viaje
    ratio_finde = dias_finde_total / duracion
    
    return round(ratio_finde, 2)

In [ ]:
PDF['ratio_finde'] = PDF.apply(lambda r: calcRatioFinde(r.ci_date, r.duration), axis=1)

 #### Subset de datos 
* loc works on labels in the index
* iloc works on the positions in the index (so it only takes integers).
* ix usually tries to behave like loc but falls back to behaving like iloc if the label is not in the index
 
http://stackoverflow.com/questions/31593201/pandas-iloc-vs-ix-vs-loc-explanation
 
### (!) CLAVE: evitar Chained Indexing
Usar loc e iloc es mejor que hacer slicing solo con [] porque usando solo [] podemos terminar trabajando con una copia del PDF en vez del original (el outcome es impredecible). Entonces los cambios que hacemos pueden no quedar guardados en el original. Usando .loc operamos directamente en el original. En general nos va a saltar un Warning llamado SettingWithCopy
 
http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

##### Slicing por una condicion

In [ ]:
PDF = PDF.loc[PDF['anticipation'] >= 0]

PDF.query('anticipation >= 0')

##### Seleccionar solo una columna y subset de filas que cumplan condiciones en más de una columna

In [ ]:
PDF.loc[(PDF["traveler_type"] == "couple") & (PDF["anticipation"] < 3), "duration"]

##### table de esa cosa bajo esas condiciones

In [ ]:
PDF.loc[(PDF["traveler_type"] == "couple") & (PDF["anticipation"] < 3), "duration"].value_counts()

- hacer una operacion sobre un campo solo cuando se cumplen ciertas condiciones

In [ ]:
PDF.loc[(PDF["traveler_type"] == "couple") & (PDF["anticipation"] < 3), "duration"] *= 1000

In [ ]:
PDF.loc[:50,['traveler_type','anticipation','duration']]

### Slice de las columnas que quiero quedarme

In [ ]:
# por nombre
PDF.loc[:,['userid','traveler_type','duration']] # es lo mismo poner PDF.ix[:,['userid','traveler_type','duration']]

# por posicion
PDF.iloc[:,[0, 15, 12]] # es lo mismo poner PDF.ix[:,[0, 15, 12]]

In [ ]:
#### Función que filtra outliers

def filterOutliers(events, features_to_filter):
    
    print events[features_to_filter].describe()
    
    for column in events:
        if column in features_to_filter:
            X_col = events[column] # se queda solo con las columnas que elegi para evaluar outliers
            # se queda solo con las obs de TODAS LAS COLUMNAS de bookings donde esas columnas cumplen la siguiente condicion.
            events = events[np.abs(X_col - X_col.mean()) <= (3 * X_col.std())] 
    
    print events[features_to_filter].describe()
    
    return events

In [ ]:
features_w_outliers = ['actions_count', 'session_count', 'search_count', 'detail_count', 'checkout_count',
                       'duration', 'anticipation']

clean_PDF = filterOutliers(PDF, features_w_outliers)

#### Sorting por una columna

In [ ]:
PDF.sort_values(by='actions_count') # puedo agregar: ascending=False, na_position='last'

#### Sorting por varias columnas

In [ ]:
PDF.sort_values(['actions_count', 'search_count'], ascending=[True, False])

#### Merge
Voy a agregarle el tax rate segun iata_dest

pd.merge:
- how: One of 'left', 'right', 'outer', 'inner'. Defaults to inner.
- suffixes: A tuple of string suffixes to apply to overlapping columns. Defaults to ('_x', '_y').

Ver: http://pandas.pydata.org/pandas-docs/stable/merging.html#database-style-dataframe-joining-merging

In [ ]:
tax_rate_table = pd.read_csv("calculo_pritax.csv", sep=',', header='infer')

In [ ]:
fixed_taxPDF = pd.merge(alldataPDF, unique_iatas, how='left', left_on=['iata_dest'], right_on=['fk_key'])

#### Me quedo con un % de sample random para test y train

In [ ]:
test = clean_PDF.sample(frac=0.1, replace=False)
train = clean_PDF.sample(frac=0.9, replace=False)